In [24]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [25]:
path = 'training_data/2020_05_04/host/'
df = pd.read_csv(path+"host_data_full.csv")

In [26]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [27]:
df = df[(df['timestamp']>= 1588542600000)&(df['timestamp']< 1588543260000)]

In [28]:
# We get a list of hosts from the previous step (here it's manual)
hosts = df['cmdb_id'].unique()
hosts

array(['os_012', 'os_020', 'os_017', 'os_008', 'os_015', 'os_002',
       'os_009', 'os_006', 'os_022', 'os_011', 'os_007', 'os_010',
       'db_010', 'db_008', 'db_007', 'db_003', 'db_001', 'os_004',
       'db_009', 'redis_006', 'os_001', 'os_003', 'os_013', 'redis_001',
       'os_016', 'os_005', 'redis_007', 'redis_010', 'redis_004',
       'redis_011', 'redis_012', 'docker_004', 'docker_008', 'redis_008',
       'db_004', 'os_021', 'os_019', 'os_014', 'os_018', 'db_013',
       'redis_003', 'redis_009', 'db_006', 'db_011', 'docker_007',
       'db_002', 'db_005', 'redis_002', 'container_002', 'container_001',
       'redis_005', 'docker_001', 'docker_002', 'db_012', 'docker_003',
       'docker_005', 'docker_006'], dtype=object)

In [29]:
# itemid,name,bomc_id,timestamp,value,cmdb_id
grouped_by_host = df[df['cmdb_id'].isin(hosts)].groupby('cmdb_id')[['name', 'value']]

In [45]:
from sklearn.neighbors import KernelDensity
from sklearn import preprocessing

anomalies = {}

def find_outlier(values):
    normalized_x = preprocessing.normalize([values])
    X = normalized_x.reshape(-1,1)
    kde = KernelDensity(kernel='tophat', bandwidth=1.0).fit(X)
    yvals = kde.score_samples(X)
    outliers = np.where(yvals < np.percentile(yvals, 1))[0]
    return outliers

for host, item in grouped_by_host:
    df_1 = grouped_by_host.get_group(host)
    grouped_by_name = df_1.groupby('name')['value'].apply(list).reset_index(name='values')
    print("Host %s has %d names"%(host, len(grouped_by_name)))
    for i in range(len(grouped_by_name)):
        row = grouped_by_name.iloc[i]
        name = row['name']
        values = row['values']
        outliers = find_outlier(values)
        if (len(outliers)):
            anomalies[host+" ---> "+name] = outliers

print("done")

Host container_001 has 1 names
Host container_002 has 1 names
Host db_001 has 43 names
Host db_002 has 43 names
Host db_003 has 46 names
Host db_004 has 43 names
Host db_005 has 43 names
Host db_006 has 43 names
Host db_007 has 46 names
Host db_008 has 43 names
Host db_009 has 43 names
Host db_010 has 45 names
Host db_011 has 45 names
Host db_012 has 45 names
Host db_013 has 45 names
Host docker_001 has 9 names
Host docker_002 has 9 names
Host docker_003 has 9 names
Host docker_004 has 9 names
Host docker_005 has 9 names
Host docker_006 has 9 names
Host docker_007 has 9 names
Host docker_008 has 9 names
Host os_001 has 50 names
Host os_002 has 50 names
Host os_003 has 50 names
Host os_004 has 50 names
Host os_005 has 50 names
Host os_006 has 50 names
Host os_007 has 50 names
Host os_008 has 50 names
Host os_009 has 50 names
Host os_010 has 50 names
Host os_011 has 50 names
Host os_012 has 50 names
Host os_013 has 50 names
Host os_014 has 50 names
Host os_015 has 50 names
Host os_016 ha

In [46]:
len(anomalies)

110